# Computing Lyapunov exponents
This script aims at computing the Lyapunov exponent for the Lorenz system in the case where the Jacobian matrix is explicitly known.

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import math as math
from tqdm import tqdm
from scipy import integrate
from scipy.linalg import expm
from sklearn.decomposition import PCA
from __future__ import division

In [3]:
#parameters of the system
rho = 28
sigma = 10
beta =8/3

#Initial condition
x0=np.array([-8, -8, 27])

def difeq(data,t):
    '''System of differential eqautions'''
    x,y,z=data
    xp=sigma*(y-x)
    yp=x*(rho-z)-y
    zp=x*y-beta*z
    return [xp,yp,zp]

In [4]:
dt=0.01
T=10000
discard_t = 100
T_total = T+discard_t
t=np.arange(0,T_total,dt)
X=integrate.odeint(difeq,x0,t)[int(discard_t):]
N=len(X)

## Computing Lyapunov exponents for known flow matrix
The flow matrix does locally describe the evolution of points in the phase space. In the case of the Lorenz system, one can easily find the flow matrix by taking the Jacobian of the Lorenz system. The implemented algorithm follows the algorithm proposed by Chen et al. in "Computing Lyapunov exponents based on the solution expression of the variational system". 

In [35]:
def get_Jacobian(pos):
    """Explicit Jacobian for Lorenz system"""
    x,y,z=pos
    J=np.array([[-sigma,sigma,0],[rho-z,-1,-x],[y,x,-beta]])
    return J

def QR_decomp(Matrix):
    '''This fct does QR decomposition of the input matrix 
    and makes sure that the diagonal element of R are positive
    (This is conventional as the QR decomposition is not unique)'''
    Qmat,Rmat = np.linalg.qr(Matrix)
    Rdiag=np.diag(Rmat)
    for k in range(len(Rdiag)):
        if Rdiag[k]<0:
            Rmat[k,:]=-Rmat[k,:]
            Qmat[:,k]=-Qmat[:,k]
    return Qmat,Rmat


def lyapexp(data,N,dt,L=50):
    #Dimension of state space
    dim=len(data.T)

    
    #Global flow
    Flow = np.identity(dim)
    Qhat = np.identity(dim)
    
    #local Jacobian at point data[0]
    J=get_Jacobian(data[0])

    #time evolution
    M=expm(dt*J)

    #Using Q,R decomposition
    Q, R = QR_decomp(M)
    
    #Storing the Lyapunov exponents
    exponents = np.zeros(len(R))
    
    for i in tqdm(range(1,N)):
        J=get_Jacobian(data[i])
        M=expm(dt*J)
        Flow = np.dot(M,Flow)
        
        #Compute Lyapunov exponent
        if i % L == 0:
            Flow = np.linalg.multi_dot([Qhat.T, Flow, Qhat])
            Q, R = QR_decomp(Flow)
            exponents=exponents+np.log(np.diag(R))
            #Compute new Qhat
            Qhat = np.dot(Qhat,Q)
            #Reset the flow
            Flow = np.identity(dim)
    #Normalization Constant
    Normal=N*dt
    return exponents/Normal


lyapexp(X,N=500010,dt=0.01,L=1)



100%|████████████████████████████████| 500009/500009 [00:30<00:00, 16389.31it/s]


array([ 9.02446646e-01, -9.35830947e-05, -1.45689924e+01])